<a href="https://colab.research.google.com/github/cocofafa/python_AI_study/blob/master/09_27_Elice_%EC%9C%A0%EA%B0%80(%EC%A3%BC%EC%84%9D_%EC%95%88%EB%8B%A8_%EB%B2%84%EC%A0%84).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_dirs = ['/usr/share/fonts/truetype/nanum', ]
font_files = fm.findSystemFonts(fontpaths=font_dirs)
for font_file in font_files:
    fm.fontManager.addfont(font_file)

plt.rcParams['font.family'] = 'NanumBarunGothic'
plt.rcParams['axes.unicode_minus']=False

In [ ]:
f18 = pd.read_csv(f'./data/과거_판매가격(주유소)_2018.csv')

In [ ]:
f18.head()

In [ ]:
# 0번 row 제거
f18 = f18.drop(0) 

In [ ]:
# 변수별 null값 확인 결과 null 없음
f18.isna().sum() 

In [ ]:
# include='all': 카테고리형 변수도 정보 제공
f18.describe(include='all') 

In [ ]:
# 기간을 datetime 형태로 변환
f18['기간'] = f18['기간'].apply(lambda x:pd.to_datetime(str(int(x)))) 

In [ ]:
# 지역 변수 중 첫 지역 구분만 컬럼 형성
region_len = f18['지역'].apply(lambda x: len(x.split())) 
print(f"min: {min(region_len)},max: {max(region_len)}")

In [ ]:
f18['지역2'] = f18['지역'].apply(lambda x:x.split()[0])
import collections
collections.Counter(f18['지역2'])

In [ ]:
# 휘발유값 0인 ROW 확인
f18.loc[f18['휘발유']==0].head(10) 

In [ ]:
f18.loc[f18['번호']=='A0010629']

In [ ]:
# 휘발유값 0인 ROW 제거
f18 = f18.loc[f18['휘발유']!=0,:]

In [ ]:
f18.describe(include='all',datetime_is_numeric=True)

In [ ]:
#주유소별 데이터 정합성 확인(7일동안 변화 없었다는 전제)
unique_count = f18.groupby('번호')[['지역','상표','셀프여부']].nunique()
unique_count.head()

In [ ]:
target = unique_count.loc[(unique_count!=1).sum(axis=1)!=0]
target

In [ ]:
f18.loc[f18['번호'].isin(target.index)]

In [ ]:
f18 = f18.loc[~f18['번호'].isin(target.index)]

In [ ]:
# 주유소별 데이터 통합
f18 = f18.groupby('번호')\
    .agg({'지역':'first','지역2':'first','상표':'first','셀프여부':'first','휘발유':'mean'})\
    .reset_index() 

In [ ]:
f18.describe(include='all')